In [7]:
import requests
import pandas as pd
import time
import os

def get_stock_valuations(ticker, start_date, end_date, api_token):
    """
    Fetch daily valuation metrics for a stock
    """
    eod_url = f'https://eodhd.com/api/eod/{ticker}.US'
    eod_params = {
        'api_token': api_token,
        'from': start_date,
        'to': end_date,
        'period': 'd',
        'fmt': 'json'
    }
    fund_url = f'https://eodhd.com/api/fundamentals/{ticker}.US'
    fund_params = {
        'api_token': api_token
    }

    try:
        price_response = requests.get(eod_url, params=eod_params)
        price_response.raise_for_status()
        prices = price_response.json()

        time.sleep(0.3)

        fund_response = requests.get(fund_url, params=fund_params)
        fund_response.raise_for_status()
        fundamentals = fund_response.json()

        valuation = fundamentals.get('Valuation', {})

        pe_ratio = valuation.get('TrailingPE')
        pb_ratio = valuation.get('PriceBookMRQ')
        ps_ratio = valuation.get('PriceSalesTTM')

        df = pd.DataFrame(prices)
        df['Date'] = pd.to_datetime(df['date']).dt.date
        df['Ticker'] = ticker
        df['Close'] = df['close']

        df['PE'] = pe_ratio
        df['PB'] = pb_ratio
        df['PS'] = ps_ratio

        result = df[['Date', 'Ticker', 'Close', 'PE', 'PB', 'PS']].copy()

        return result

    except Exception as e:
        print(f"Error: {e}")
        return None

# This is a paid personal API, so I didn't put mine here. You can put your
# own API here and run this code.
API_TOKEN = ''
START_DATE = '2023-01-01'
END_DATE = '2023-12-31'
OUTPUT_DIR = 'stock_valuations_2023'

STOCKS = [
    'AAPL', 'AMZN', 'ASML', 'AVGO', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA',
    'PYPL', 'COST', 'EBAY', 'JD', 'PDD', 'NFLX', 'ENPH', 'TMUS', 'FAST', 'MRNA',
    'CSCO'
]

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Fetching PE, PB, PS data for {len(STOCKS)} stocks")
print(f"Period: {START_DATE} to {END_DATE}\n")

all_data = []
success_count = 0
fail_count = 0

for i, ticker in enumerate(STOCKS, 1):
    print(f"[{i}/{len(STOCKS)}] {ticker}...", end=" ")

    try:
        df = get_stock_valuations(ticker, START_DATE, END_DATE, API_TOKEN)

        if df is not None and len(df) > 0:
            individual_file = os.path.join(OUTPUT_DIR, f'{ticker}_valuations_2023.csv')
            df.to_csv(individual_file, index=False)

            all_data.append(df)
            success_count += 1

            print(f"✅ {len(df)} days - PE:{df['PE'].iloc[0]:.2f}, PB:{df['PB'].iloc[0]:.2f}, PS:{df['PS'].iloc[0]:.2f}")
        else:
            print("No data")
            fail_count += 1

    except Exception as e:
        print(f"Error: {str(e)}")
        fail_count += 1

    time.sleep(1)

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values(['Date', 'Ticker'])

    combined_csv = os.path.join(OUTPUT_DIR, 'ALL_STOCKS_valuations_2023.csv')

    combined_df.to_csv(combined_csv, index=False)

    print(f"\n{'='*60}")
    print(f"Summary:")
    print(f"  Success: {success_count}/{len(STOCKS)}")
    print(f"  Failed: {fail_count}/{len(STOCKS)}")
    print(f"  Total rows: {len(combined_df)}")
    print(f"\nFiles saved:")
    print(f"  Individual: {OUTPUT_DIR}/[TICKER]_valuations_2023.csv")
    print(f"  Combined CSV: {combined_csv}")
    print(f"{'='*60}")

    print(f"\nSample data:")
    print(combined_df.head(10))
else:
    print("\nNo data retrieved")


Fetching PE, PB, PS data for 20 stocks
Period: 2023-01-01 to 2023-01-05

[1/20] AAPL... ✅ 3 days - PE:37.32, PB:56.94, PS:9.94
[2/20] AMZN... ✅ 3 days - PE:32.47, PB:6.72, PS:3.55
[3/20] ASML... ✅ 3 days - PE:38.91, PB:20.05, PS:13.25
[4/20] AVGO... ✅ 3 days - PE:100.32, PB:24.53, PS:30.75
[5/20] GOOGL... ✅ 3 days - PE:31.71, PB:9.97, PS:10.09
[6/20] META... ✅ 3 days - PE:29.76, PB:8.40, PS:8.96
[7/20] MSFT... ✅ 3 days - PE:34.36, PB:9.78, PS:12.22
[8/20] NVDA... ✅ 3 days - PE:45.26, PB:36.71, PS:23.73
[9/20] TSLA... ✅ 3 days - PE:311.64, PB:17.85, PS:15.82
[10/20] PYPL... ✅ 3 days - PE:12.51, PB:2.91, PS:1.81
[11/20] COST... ✅ 3 days - PE:49.19, PB:14.01, PS:1.44
[12/20] EBAY... ✅ 3 days - PE:18.29, PB:7.87, PS:3.52
[13/20] JD... ✅ 3 days - PE:9.97, PB:1.29, PS:0.03
[14/20] PDD... ✅ 3 days - PE:12.06, PB:3.01, PS:0.40
[15/20] NFLX... ✅ 3 days - PE:41.77, PB:16.97, PS:9.79
[16/20] ENPH... ✅ 3 days - PE:21.40, PB:3.83, PS:2.70
[17/20] TMUS... ✅ 3 days - PE:20.20, PB:3.85, PS:2.75
[18/20